In [1]:
! pip install -q ragstack-ai datasets

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torchvision 0.18.0 requires torch==2.3.0, but you have torch 2.2.1 which is incompatible.


In [1]:
import cassio

In [5]:
import os
from getpass import getpass

# Enter your settings for Astra DB and OpenAI:
os.environ["ASTRA_DB_API_ENDPOINT"] ="https://96111f6a-d1cc-4c86-8115-c7ac9cde8794-us-east-2.apps.astra.datastax.com"
os.environ["ASTRA_DB_APPLICATION_TOKEN"] ="AstraCS:SBlmevhiETPSEPyDoUsvAJDL:dbad27b53e2665c10c8905aaca0cffd6f12af9b1c5407ea19e1757bb5ec3034f"

In [4]:
import os 
from dotenv import load_dotenv 
load_dotenv()
groq_api_key = os.environ["GROQ_API_KEY"]
from langchain_groq import ChatGroq
llm = ChatGroq(groq_api_key= groq_api_key,
               model="mixtral-8x7b-32768" )

Create RAG Pipeline

In [6]:
from langchain_astradb import AstraDBVectorStore
from langchain_community.embeddings import HuggingFaceEmbeddings


# Configure your embedding model and vector store
embedding = HuggingFaceEmbeddings()
vstore = AstraDBVectorStore(
    collection_name="test",
    embedding=embedding,
    token=os.getenv("ASTRA_DB_APPLICATION_TOKEN"),
    api_endpoint=os.getenv("ASTRA_DB_API_ENDPOINT"),
)
print("Astra vector store configured")

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.6k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

/home/codespace/.python/current/lib/python3.10/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


...

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

Astra vector store configured


In [8]:
from datasets import load_dataset

# Load a sample dataset
philo_dataset = load_dataset("datastax/philosopher-quotes")["train"]
print("An example entry:")
print(philo_dataset[10])

An example entry:
{'author': 'aristotle', 'quote': 'Fortune favours the bold.', 'tags': None}


In [9]:
philo_dataset

Dataset({
    features: ['author', 'quote', 'tags'],
    num_rows: 450
})

In [10]:
from langchain.schema import Document

# Constructs a set of documents from your data. Documents can be used as inputs to your vector store.
docs = []
for entry in philo_dataset:
    metadata = {"author": entry["author"]}
    if entry["tags"]:
        # Add metadata tags to the metadata dictionary
        for tag in entry["tags"].split(";"):
            metadata[tag] = "y"
    # Create a LangChain document with the quote and metadata tags
    doc = Document(page_content=entry["quote"], metadata=metadata)
    docs.append(doc)

In [11]:
docs

[Document(page_content="True happiness comes from gaining insight and growing into your best possible self. Otherwise all you're having is immediate gratification pleasure, which is fleeting and doesn't grow you as a person.", metadata={'author': 'aristotle', 'knowledge': 'y'}),
 Document(page_content='The roots of education are bitter, but the fruit is sweet.', metadata={'author': 'aristotle', 'education': 'y', 'knowledge': 'y'}),
 Document(page_content='Before you heal the body you must first heal the mind', metadata={'author': 'aristotle', 'ethics': 'y'}),
 Document(page_content='The proof that you know something is that you are able to teach it', metadata={'author': 'aristotle', 'education': 'y', 'knowledge': 'y'}),
 Document(page_content='Those who are not angry at the things they should be angry at are thought to be fools, and so are those who are not angry in the right way, at the right time, or with the right persons.', metadata={'author': 'aristotle'}),
 Document(page_content=

In [12]:
# Create embeddings by inserting your documents into the vector store.
inserted_ids = vstore.add_documents(docs)
print(f"\nInserted {len(inserted_ids)} documents.")


Inserted 450 documents.


In [13]:
# Checks your collection to verify the documents are embedded.
print(vstore.astra_db.collection("test").find())

{'data': {'documents': [{'_id': '65221ccec80d4796b2a3f9b8919a0fcf', 'content': 'It strikes everyone in beginning to form an acquaintance with the treasures of Indian literature that a land so rich in intellectual products and those of the profoundest order of thought.', '$vector': [0.017279818654060364, 0.05754787474870682, -0.030385635793209076, 0.019751811400055885, -0.029344476759433746, 0.0022380631417036057, -0.05656406283378601, 0.0005953526706434786, 0.015085971914231777, -0.013662909157574177, -0.02188067138195038, 0.024292733520269394, -0.007786594331264496, -0.01600143499672413, 0.014207159169018269, -0.003945434466004372, 0.01747099682688713, 0.006558879744261503, -0.029442952945828438, 0.04504399374127388, -0.013051614165306091, 0.03238673880696297, -0.03203541412949562, 0.00999528355896473, 0.014393499121069908, -0.05030684173107147, 0.029269367456436157, -0.0031773834489285946, 0.021783441305160522, -0.0891299843788147, 0.000127306193462573, 0.0015090061351656914, 0.02084

In [15]:
from langchain.prompts import ChatPromptTemplate
from langchain.schema.output_parser import StrOutputParser
from langchain.schema.runnable import RunnablePassthrough

retriever = vstore.as_retriever(search_kwargs={"k": 3})

prompt_template = """
Answer the question based only on the supplied context. If you don't know the answer, say you don't know the answer.
Context: {context}
Question: {question}
Your answer:
"""
prompt = ChatPromptTemplate.from_template(prompt_template)
model = llm

chain = (
    {"context": retriever, "question": RunnablePassthrough()}
    | prompt
    | model
    | StrOutputParser()
)

chain.invoke("In the given context, what is the most important to allow the brain and provide me the tags?")

"The most important thing to allow for the brain, according to Schopenhauer, is the full measure of sleep that is required to restore it. The relevant tags for this statement are 'schopenhauer', 'ethics', and 'knowledge'."